In [1]:
import pandas as pd
import numpy as np

In [3]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])

In [5]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five

1. Using the daily stock data from 2010, return an array of the unique trading days in 2010.

In [21]:
df2010.loc[:, 'retdate'].unique()

array(['2010-04-07T00:00:00.000000000', '2010-04-13T00:00:00.000000000',
       '2010-06-11T00:00:00.000000000', '2010-11-23T00:00:00.000000000',
       '2010-02-01T00:00:00.000000000', '2010-11-22T00:00:00.000000000',
       '2010-08-23T00:00:00.000000000', '2010-06-10T00:00:00.000000000',
       '2010-06-04T00:00:00.000000000', '2010-04-12T00:00:00.000000000',
       '2010-04-06T00:00:00.000000000', '2010-08-09T00:00:00.000000000',
       '2010-02-02T00:00:00.000000000', '2010-11-08T00:00:00.000000000',
       '2010-02-16T00:00:00.000000000', '2010-02-17T00:00:00.000000000',
       '2010-11-09T00:00:00.000000000', '2010-02-03T00:00:00.000000000',
       '2010-08-20T00:00:00.000000000', '2010-06-07T00:00:00.000000000',
       '2010-04-05T00:00:00.000000000', '2010-04-15T00:00:00.000000000',
       '2010-04-01T00:00:00.000000000', '2010-04-29T00:00:00.000000000',
       '2010-06-17T00:00:00.000000000', '2010-06-03T00:00:00.000000000',
       '2010-08-18T00:00:00.000000000', '2010-08-30

2. Return the symbol, return date and the dollar volume traded for the highest dollar volume traded stocks in 2010 on the NYSE.

In [16]:
(df2010.loc[(df2010.loc[:, 'exch'] == 'NYSE'), :].
assign(dol_vol = df2010.loc[:, 'cls']*df2010.loc[:, 'vol']).
nlargest(1, 'dol_vol').loc[:, ['symb', 'retdate', 'dol_vol']])

,symb,retdate,dol_vol
553212,BRK.B,2010-02-12,2.415266e+10


3. Using the 2010 data, return the stock (symbol only) with the largest volume on Jan 11th that also appears on Dec 1st.

In [27]:
(df2010.loc[
    (df2010.loc[:, 'symb'].isin(df2010.loc[(df2010.loc[:, 'retdate'] == '2010-12-01'), 'symb'].unique()) &
    (df2010.loc[:, 'retdate'] == '2010-01-11')), :].
    nlargest(1, 'vol').
    loc[:, ['symb']])

,symb
468965,F


In [17]:
df2010.retdate

0        2010-04-07
1        2010-04-07
2        2010-04-07
3        2010-04-07
4        2010-04-07
            ...    
816061   2010-01-25
816062   2010-01-25
816063   2010-01-25
816064   2010-01-25
816065   2010-01-25
Name: retdate, Length: 816066, dtype: datetime64[ns]